In [153]:
import os
from typing import List
from datetime import datetime

import numpy as np
import pandas as pd

TESTING = True

DATA_ENVIRONMENT: str = "testing" if TESTING else "production"
DATA_FOLDER: str = f"data/{DATA_ENVIRONMENT}"
DATETIME_FORMAT_STRING: List[str] = ["%Y-%m-%d %H:%M:%S.%f", "%Y-%m-%d %H:%M:%S"]

def str2timestamp(date: str) -> int:
    try:
        return int(datetime.strptime(date, DATETIME_FORMAT_STRING[0]).timestamp())
    except ValueError:
        try:
            return int(datetime.strptime(date, DATETIME_FORMAT_STRING[1]).timestamp())
        except ValueError as exc:
            raise ValueError(f"Could not convert {date} to timestamp.") from exc


def read_data(name: str) -> pd.DataFrame:
    """
    Reads data from a csv file.
    """

    return pd.read_csv(os.path.join(os.getcwd(), DATA_FOLDER, name), header=0)

In [86]:
first: str = "2022-04-01 12:44:10.315"
first_ts: int = str2timestamp(first)

In [87]:
df: pd.DataFrame = read_data("place1Msample.csv")
df["timestamp"] = df["time"] + (first_ts * 1000)
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)
df

,time,user_id,x,y,color,mod,timestamp
0,0,0,42,42,15,0,2022-04-01 15:44:10+00:00
1,12356,1,999,999,22,0,2022-04-01 15:44:22.356000+00:00
2,16311,2,44,42,26,0,2022-04-01 15:44:26.311000+00:00
3,21388,3,2,2,29,0,2022-04-01 15:44:31.388000+00:00
4,34094,4,23,23,26,0,2022-04-01 15:44:44.094000+00:00
...,...,...,...,...,...,...,...
999994,6519028,389634,914,910,9,0,2022-04-01 17:32:49.028000+00:00
999995,6519029,63770,446,803,0,0,2022-04-01 17:32:49.029000+00:00
999996,6519035,389635,757,450,9,0,2022-04-01 17:32:49.035000+00:00
999997,6519035,389636,439,151,9,0,2022-04-01 17:32:49.035000+00:00


In [154]:
df_by_user = df.groupby(by=["user_id"]).agg({ "timestamp": ["min", "max"], "x": "count" })
df_by_user["max_draw"] = np.floor(1 + ((df_by_user["timestamp"]["max"] - df_by_user["timestamp"]["min"]).dt.total_seconds()) / (60 * 5))

In [155]:
df_by_user[df_by_user[("x", "count")] == 18]

timestamp                                   \
                                     min                              max   
user_id                                                                     
23      2022-04-01 16:03:45.576000+00:00 2022-04-01 17:32:23.835000+00:00   
41      2022-04-01 16:03:51.660000+00:00 2022-04-01 17:30:52.444000+00:00   
45      2022-04-01 16:03:52.941000+00:00 2022-04-01 17:31:09.811000+00:00   
56      2022-04-01 16:03:54.187000+00:00 2022-04-01 17:29:48.047000+00:00   
98      2022-04-01 16:03:58.664000+00:00 2022-04-01 17:32:28.843000+00:00   
...                                  ...                              ...   
3898    2022-04-01 16:05:49.266000+00:00 2022-04-01 17:32:13.630000+00:00   
4404    2022-04-01 16:06:01.720000+00:00 2022-04-01 17:30:18.232000+00:00   
4504    2022-04-01 16:06:04.060000+00:00 2022-04-01 17:32:23.447000+00:00   
4666    2022-04-01 16:06:07.329000+00:00 2022-04-01 17:31:56.906000+00:00   
7561    2022-04-01 16:06:59.555000+00:00 2022-04-01 17:32:22.425000+00:00   

            x max_draw  
        count           
user_id                 
23         18     18.0  
41         18     18.0  
45         18     18.0  
56         18     18.0  
98         18     18.0  
...       ...      ...  
3898       18     18.0  
4404       18     17.0  
4504       18     18.0  
4666       18     18.0  
7561       18     18.0  

[82 rows x 4 columns]